In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./freesound-audio-tagging/"))
import librosa
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import skew
SAMPLE_RATE = 44100

# from sklearn.model_selection import KFold, RepeatedKFold
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()
import scipy
data_path = './freesound-audio-tagging/'
ss = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))

['audio_train.zip', 'audio_train', 'audio_test', 'sample_submission.csv', 'audio_test.zip', 'train.csv']


In [4]:
# loading data
audio_train_files = os.listdir('./freesound-audio-tagging/audio_train')
audio_test_files = os.listdir('./freesound-audio-tagging/audio_test')

train = pd.read_csv('./freesound-audio-tagging/train.csv')
submission = pd.read_csv('./freesound-audio-tagging/sample_submission.csv')

In [5]:
# function from EDA kernel: https://www.kaggle.com/codename007/a-very-extensive-freesound-exploratory-analysis
def clean_filename(fname, string):   
    file_name = fname.split('/')[1]
    if file_name[:2] == '__':        
        file_name = string + file_name
    return file_name

#returns mfcc features with mean and standard deviation along time
def get_mfcc(name, path):
    b, _ = librosa.core.load(path + name, sr = SAMPLE_RATE)
    assert _ == SAMPLE_RATE
    try:
        ft1 = librosa.feature.mfcc(b, sr = SAMPLE_RATE, n_mfcc=20)
        ft2 = librosa.feature.zero_crossing_rate(b)[0]
        ft3 = librosa.feature.spectral_rolloff(b)[0]
        ft4 = librosa.feature.spectral_centroid(b)[0]
        ft5 = librosa.feature.spectral_contrast(b)[0]
        ft6 = librosa.feature.spectral_bandwidth(b)[0]
        ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.min(ft1, axis = 1)))
        ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.min(ft2)))
        ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.min(ft3)))
        ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.min(ft4)))
        ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.min(ft5)))
        ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.max(ft6)))
        return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc)))
    except:
        print('bad file')
        return pd.Series([0]*125)

In [6]:
# preparing data
train_data = pd.DataFrame()
train_data['fname'] = train['fname']
test_data = pd.DataFrame()
test_data['fname'] = audio_test_files

train_data = train_data['fname'].progress_apply(get_mfcc, path='./freesound-audio-tagging/audio_train/')
print('done loading train mfcc')
test_data = test_data['fname'].progress_apply(get_mfcc, path='./freesound-audio-tagging/audio_test/')
print('done loading test mfcc')

train_data['fname'] = train['fname']
test_data['fname'] = audio_test_files
train_data['label'] = train['label']
test_data['label'] = np.zeros((len(audio_test_files)))

  0%|          | 2/9400 [00:00<11:40, 13.42it/s]

done loading train mfcc


  9%|▉         | 864/9400 [02:37<21:18,  6.68it/s]  

bad file


 27%|██▋       | 2516/9400 [07:58<18:30,  6.20it/s]  

bad file


 56%|█████▌    | 5246/9400 [16:42<09:31,  7.27it/s]

bad file


100%|██████████| 9400/9400 [30:13<00:00,  5.99it/s]  

done loading test mfcc


In [8]:
train_data.to_csv(data_path + 'train_feature_0.csv', index=False)
test_data.to_csv(data_path + 'test_feature_0.csv', index=False)
train_weight = [(x * 0.3 + 0.7) for x in train['manually_verified'].values]

In [9]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,fname,label
0,-446.838467,52.372300,-53.377823,-3.740264,-8.318184,-11.119042,-14.467405,-10.285843,-10.966904,-2.670278,...,0.937812,33.705153,7.055138,1989.168387,775.044969,0.714498,3543.959962,3543.959962,00044347.wav,Hi-hat
1,-375.754678,151.572502,-41.933635,-0.165071,5.955898,-12.535086,-13.192531,-13.657687,-9.174413,-19.668802,...,-1.262496,60.328617,6.676251,906.662232,506.447373,3.305549,3388.692162,3388.692162,001ca53d.wav,Saxophone
2,-658.236035,102.368917,20.626247,9.108918,-4.891736,7.259463,-1.862530,1.661778,5.499974,5.338336,...,0.628735,35.780019,8.280023,2840.134501,611.530727,-0.202585,3631.676275,3631.676275,002d256b.wav,Trumpet
3,-770.905450,16.077204,-3.556132,-10.957762,-19.071291,-9.632298,4.279781,14.633798,19.459229,12.089039,...,0.818964,36.509611,13.943471,2758.526365,772.741940,-0.909257,3553.924270,3553.924270,0033e230.wav,Glockenspiel
4,-343.603965,137.873119,-28.701815,8.593180,2.751699,-0.301433,-9.502459,-3.774746,13.056789,7.197487,...,-0.279604,35.938348,8.232091,1639.312250,657.051222,1.086833,3113.026574,3113.026574,00353774.wav,Cello


In [27]:
train_data = pd.read_csv(data_path + 'train_feature_0.csv')
test_data = pd.read_csv(data_path + 'test_feature_0.csv')
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,fname,label
0,-446.838467,52.372300,-53.377823,-3.740264,-8.318184,-11.119042,-14.467405,-10.285843,-10.966904,-2.670278,...,0.937812,33.705153,7.055138,1989.168387,775.044969,0.714498,3543.959962,3543.959962,00044347.wav,Hi-hat
1,-375.754678,151.572502,-41.933635,-0.165071,5.955898,-12.535086,-13.192531,-13.657687,-9.174413,-19.668802,...,-1.262496,60.328617,6.676251,906.662232,506.447373,3.305549,3388.692162,3388.692162,001ca53d.wav,Saxophone
2,-658.236035,102.368917,20.626247,9.108918,-4.891736,7.259463,-1.862530,1.661778,5.499974,5.338336,...,0.628735,35.780019,8.280023,2840.134501,611.530727,-0.202585,3631.676275,3631.676275,002d256b.wav,Trumpet
3,-770.905450,16.077204,-3.556132,-10.957762,-19.071291,-9.632298,4.279781,14.633798,19.459229,12.089039,...,0.818964,36.509611,13.943471,2758.526365,772.741940,-0.909257,3553.924270,3553.924270,0033e230.wav,Glockenspiel
4,-343.603965,137.873119,-28.701815,8.593180,2.751699,-0.301433,-9.502459,-3.774746,13.056789,7.197487,...,-0.279604,35.938348,8.232091,1639.312250,657.051222,1.086833,3113.026574,3113.026574,00353774.wav,Cello


In [17]:
# Features from LightGBM baseline kernel: https://www.kaggle.com/opanichev/lightgbm-baseline
# MAPk from https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])


def extract_features(files, path):
    features = {}

    cnt = 0
    for f in tqdm(files):
        features[f] = {}

        fs, data = scipy.io.wavfile.read(os.path.join(path, f))

        abs_data = np.abs(data)
        diff_data = np.diff(data)

        def calc_part_features(data, n=2, prefix=''):
            f_i = 1
            step = len(data) // n
            for i in range(n):
                features[f]['{}mean_{}_{}'.format(prefix, f_i, n)] = np.mean(data[i * step:(i + 1) * step])
                features[f]['{}std_{}_{}'.format(prefix, f_i, n)] = np.std(data[i * step:(i + 1) * step])
                features[f]['{}min_{}_{}'.format(prefix, f_i, n)] = np.min(data[i * step:(i + 1) * step])
                features[f]['{}max_{}_{}'.format(prefix, f_i, n)] = np.max(data[i * step:(i + 1) * step])
                f_i += 1

        features[f]['len'] = len(data)
        if features[f]['len'] > 0:
            n = 1
            calc_part_features(data, n=n)
            calc_part_features(abs_data, n=n, prefix='abs_')
            calc_part_features(diff_data, n=n, prefix='diff_')

            n = 2
            calc_part_features(data, n=n)
            calc_part_features(abs_data, n=n, prefix='abs_')
            calc_part_features(diff_data, n=n, prefix='diff_')

            n = 3
            calc_part_features(data, n=n)
            calc_part_features(abs_data, n=n, prefix='abs_')
            calc_part_features(diff_data, n=n, prefix='diff_')


        cnt += 1

        # if cnt >= 1000:
        #     break

    features = pd.DataFrame(features).T.reset_index()
    features.rename(columns={'index': 'fname'}, inplace=True)
    
    return features

path = os.path.join(data_path, 'audio_train')
train_files = train.fname.values
train_features = extract_features(train_files, path)

path = os.path.join(data_path, 'audio_test')
test_files = ss.fname.values
test_features = extract_features(test_files, path)

100%|██████████| 9400/9400 [02:01<00:00, 77.61it/s]


In [18]:
train_data = train_data.merge(train_features, on='fname', how='left')
test_data = test_data.merge(test_features, on='fname', how='left')
train_data.head()


,0,1,2,3,4,5,6,7,8,9,...,min_1_3,min_2_2,min_2_3,min_3_3,std_1_1,std_1_2,std_1_3,std_2_2,std_2_3,std_3_3
0,-446.838467,52.372300,-53.377823,-3.740264,-8.318184,-11.119042,-14.467405,-10.285843,-10.966904,-2.670278,...,-18601,-221,-615,-126,1206.881456,1706.545234,2088.675390,28.793127,83.189913,14.249761
1,-375.754678,151.572502,-41.933635,-0.165071,5.955898,-12.535086,-13.192531,-13.657687,-9.174413,-19.668802,...,-12593,-9359,-9359,-7484,3196.572491,3495.882224,3593.781554,2866.174942,3066.586209,2887.041732
2,-658.236035,102.368917,20.626247,9.108918,-4.891736,7.259463,-1.862530,1.661778,5.499974,5.338336,...,-636,-53,-132,-33,65.608624,91.744135,110.228966,13.855835,24.067880,13.421382
3,-770.905450,16.077204,-3.556132,-10.957762,-19.071291,-9.632298,4.279781,14.633798,19.459229,12.089039,...,-8345,-25,-83,-5,259.141991,366.449985,448.192027,4.846361,24.215311,1.041946
4,-343.603965,137.873119,-28.701815,8.593180,2.751699,-0.301433,-9.502459,-3.774746,13.056789,7.197487,...,-12829,-8860,-10921,-5577,3005.821294,3895.473242,3875.704209,1701.234020,3355.832221,905.514833


In [19]:
print(train_data.columns[127:-1])

Index([u'abs_max_1_1', u'abs_max_1_2', u'abs_max_1_3', u'abs_max_2_2',
       u'abs_max_2_3', u'abs_max_3_3', u'abs_mean_1_1', u'abs_mean_1_2',
       u'abs_mean_1_3', u'abs_mean_2_2', u'abs_mean_2_3', u'abs_mean_3_3',
       u'abs_min_1_1', u'abs_min_1_2', u'abs_min_1_3', u'abs_min_2_2',
       u'abs_min_2_3', u'abs_min_3_3', u'abs_std_1_1', u'abs_std_1_2',
       u'abs_std_1_3', u'abs_std_2_2', u'abs_std_2_3', u'abs_std_3_3',
       u'diff_max_1_1', u'diff_max_1_2', u'diff_max_1_3', u'diff_max_2_2',
       u'diff_max_2_3', u'diff_max_3_3', u'diff_mean_1_1', u'diff_mean_1_2',
       u'diff_mean_1_3', u'diff_mean_2_2', u'diff_mean_2_3', u'diff_mean_3_3',
       u'diff_min_1_1', u'diff_min_1_2', u'diff_min_1_3', u'diff_min_2_2',
       u'diff_min_2_3', u'diff_min_3_3', u'diff_std_1_1', u'diff_std_1_2',
       u'diff_std_1_3', u'diff_std_2_2', u'diff_std_2_3', u'diff_std_3_3',
       u'len', u'max_1_1', u'max_1_2', u'max_1_3', u'max_2_2', u'max_2_3',
       u'max_3_3', u'mean_1_1', u'mea

In [20]:

train_data.to_csv(data_path + 'train_feature_1.csv', index=False)
test_data.to_csv(data_path + 'test_feature_1.csv', index=False)

In [33]:
train_data = pd.read_csv(data_path + 'train_feature_1.csv')
test_data = pd.read_csv(data_path + 'test_feature_1.csv')
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,min_1_3,min_2_2,min_2_3,min_3_3,std_1_1,std_1_2,std_1_3,std_2_2,std_2_3,std_3_3
0,-446.838467,52.372300,-53.377823,-3.740264,-8.318184,-11.119042,-14.467405,-10.285843,-10.966904,-2.670278,...,-18601,-221,-615,-126,1206.881456,1706.545234,2088.675390,28.793127,83.189913,14.249761
1,-375.754678,151.572502,-41.933635,-0.165071,5.955898,-12.535086,-13.192531,-13.657687,-9.174413,-19.668802,...,-12593,-9359,-9359,-7484,3196.572491,3495.882224,3593.781554,2866.174942,3066.586209,2887.041732
2,-658.236035,102.368917,20.626247,9.108918,-4.891736,7.259463,-1.862530,1.661778,5.499974,5.338336,...,-636,-53,-132,-33,65.608624,91.744135,110.228966,13.855835,24.067880,13.421382
3,-770.905450,16.077204,-3.556132,-10.957762,-19.071291,-9.632298,4.279781,14.633798,19.459229,12.089039,...,-8345,-25,-83,-5,259.141991,366.449985,448.192027,4.846361,24.215311,1.041946
4,-343.603965,137.873119,-28.701815,8.593180,2.751699,-0.301433,-9.502459,-3.774746,13.056789,7.197487,...,-12829,-8860,-10921,-5577,3005.821294,3895.473242,3875.704209,1701.234020,3355.832221,905.514833


In [34]:
#Functions from LightGBM baseline: https://www.kaggle.com/opanichev/lightgbm-baseline
# Construct features set
X = train_data.drop(['label', 'fname'], axis=1)
feature_names = list(X.columns)
X = X.values
labels = np.sort(np.unique(train_data.label.values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
y = np.array([c2i[x] for x in train_data.label.values])

In [35]:
#fitting xgboost on the dataset
X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(X, y, np.array(train_weight), test_size=0.2, random_state=10, shuffle = True)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape, w_train.shape, w_val.shape)
clf = XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=150,
                    n_jobs=-1, random_state=0, reg_alpha=0.2, 
                    colsample_bylevel=0.7, colsample_bytree=0.7)
clf.fit(X_train, y_train, sample_weight=w_train)
print(accuracy_score(clf.predict(X_val), y_val))
#more functions from LightGBM baseline: https://www.kaggle.com/opanichev/lightgbm-baseline
def proba2labels(preds, i2c, k=3):
    ans = []
    ids = []
    for p in preds:
        idx = np.argsort(p)[::-1]
        ids.append([i for i in idx[:k]])
        ans.append(' '.join([i2c[i] for i in idx[:k]]))

    return ans, ids

((7578, 198), (1895, 198), (7578,), (1895,), (7578,), (1895,))
0.7245382585751979


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
#fitting on the entire data

clf.fit(X, y, sample_weight=train_weight)
str_preds, _ = proba2labels(clf.predict_proba(test_data.drop(['label', 'fname'], axis = 1).values), i2c, k=3)
# Prepare submission
subm = pd.DataFrame()
subm['fname'] = audio_test_files
subm['label'] = str_preds
subm.to_csv('submission_02_180407.csv', index=False)